# 조건 빈도 분포
- 말뭉치에 속한 텍스트들을 카테고리별로 구분할 경우
    - 말뭉치 전체에서 단어 빈도 분포가 아니라 각 카테고리별로 단어 빈도 분포를 구할 수 있습니다.
- 이처럼 조건 빈도 분포란 다양한 조건에 대한 빈도 분포입니다. 

All rights reserved, 2021, By Youn-Sik Hong. 수업 목적으로만 활용 가능.

- 참고 사이트 : https://www.nltk.org/book/ch02.html
    - nltk book 2.Accessing Text Corpora and Lexical Resources 내용을 참고해서 자료를 만듦. 
    - nltk book 2장 예제 인용.

In [ ]:
import nltk

### 조건빈도 = (condition, event)
- 조건 빈도 분포란 조건에 따른 사건(event)을 서로 쌍으로 묶는 것입니다. 
- 아래 예처럼, 단어만 따로 처리하는 대신, 단어와 카테고리를 쌍으로 묶어 처리하는 것입니다. 
    - 'The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...
    - ('news', 'The'), ('news', 'Fulton'), ('news', 'County'), ...

### 카테고리별 단어 출현 빈도
- 각 쌍은 (condition, event)의 형태입니다. 
- 전체 Brown 말뭉치를 조건빈도분포로 구하면, 
    - 15개 조건(=카테고리 수)과 1,161,192 사건(=단어 수)이 있으며, 이들을 쌍으로 묶게 됩니다.

In [ ]:
from nltk.corpus import brown

cfd = nltk.ConditionalFreqDist(
          (genre, word)
          for genre in brown.categories()
          for word in brown.words(categories=genre))

In [ ]:
print(len(cfd.conditions()), cfd.conditions())

- 좀 더 일반적인 응용 예를 보기 위해 카테고리를 2개(news, romance)로 제한한 조건빈도분포를 구해 보겠습니다. 
- (condition, event) 쌍은 튜플 타입의 원소를 갖는 리스트(**genre_word**)로 나타낼 수 있습니다.
    - 파이썬에서 튜플은 괄호를 사용하여 원소들을 묶습니다. : (genre, word)
    - 반면 리스트는 대괄호(bracket)을 사용하여 원소들을 묶죠. 

In [ ]:
genre_word = [(genre, word) 
               for genre in ['news', 'romance'] 
               for word in brown.words(categories=genre)] 

print(len(genre_word), type(genre_word))
print(genre_word[:4])
print(genre_word[-4:])

In [ ]:
cfd = nltk.ConditionalFreqDist(genre_word)
print(cfd.conditions())

- 전체 170,576개 튜플 원소 중, news 카테고리에는 100,554개, romance 카테고리에는 70,022개 튜플이 있습니다.
    - 단어 빈도 분포를 구하면 news 카테고리에는 14,394개 빈도 샘플이 있으며,
    - romance 카테고리에는 8,452개 빈도 샘플이 있습니다.   

In [ ]:
print(cfd['news'])
print(cfd['romance'])
print(len(cfd['news'].most_common()))
print(len(cfd['romance'].most_common()))
print(cfd['news'].most_common(9))
print(cfd['romance'].most_common(9))

news 카테고리에서 단어 will의 빈도, romance 카테고리에서는 단어 could의 빈도를 비교해 보겠습니다.

In [ ]:
print(cfd['news']['will'], cfd['romance']['will'])
print(cfd['news']['could'], cfd['romance']['could'])

- Brown 말뭉치에서 news와 romance 카테고리에 대해 조금 더 분석해볼까요...
    - 일주일 중 어느 요일이 '뉴스'로써 가치가 있을까요... 결과는 일, 토, 금 순입니다.
    - 어느 요일이 가장 '로맨틱'한지 알아볼까요... 역시 결과는 일, 토, 금 순입니다.

In [ ]:
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
cfd = nltk.ConditionalFreqDist(
            (genre, day) 
            for genre in ['news', 'romance'] 
            for word in brown.words(categories=genre)
            for day in days 
            if day == word)
cfd.tabulate(samples=days, cumulative=True)

### 조건빈도분포 시각화: 테이블 출력 
- 미국 대통령 취임연설 말뭉치(inaugural)를 사용해 2개 단어(america, citizen)에 대해 
    - 년도별 취임연설에서 2개 단어 빈도가 어떻게 변화했는지 2-1.nltk 말뭉치 활용에서 이미 확인했었습니다.

In [ ]:
from nltk.corpus import inaugural

cfd = nltk.ConditionalFreqDist(
          (target, fileid[:4]) #1865-Lincoln.txt에서 앞 4글자(년도)
          for fileid in inaugural.fileids()
              for w in inaugural.words(fileid)
                  for target in ['america', 'citizen'] 
          if w.lower().startswith(target)) #소문자변환 
cfd.plot()

이번에는 이를 표로 출력해, 취임연설에서 2개 단어 빈도 변화를 연도별로 정확히(?) 확인해 보겠습니다.

In [ ]:
year_list = [fileid[:4] for fileid in inaugural.fileids()]
cfd.tabulate(conditions=['america', 'citizen'],
             samples=year_list[:20], cumulative=True)

세계인권선언(udhr) 말뭉치에서 라틴계 언어의 단어 길이를 비교해 보겠습니다.

In [ ]:
from nltk.corpus import udhr

languages = ['Chickasaw', 'English', 'German_Deutsch',
     'Greenlandic_Inuktikut', 'Hungarian_Magyar', 'Ibibio_Efik']

cfd = nltk.ConditionalFreqDist(
          (lang, len(word)) 
          for lang in languages
          for word in udhr.words(lang + '-Latin1'))

In [ ]:
cfd.tabulate(conditions=languages,
             samples=range(10), cumulative=True)

- 비교 대상 언어를 영어와 독일어로 제한해 보겠습니다. 
    - 영단어 1,638개가 9개 이하 문자를 갖습니다. 
    - 독일어가 영어에 비해 단어 수가 적습니다(어휘 다양성이 낮다고 추측할 수 있습니다). 
    - 물론 샘플 자체가 적어 단순한 추측에 지나지 않습니다.

In [ ]:
cfd.tabulate(conditions=['English', 'German_Deutsch'],
             samples=range(10), cumulative=True)

아시아권 언어도 비교해 보겠습니다.

In [ ]:
languages = ['Punjabi_Panjabi', 'Hawaiian', 'Greek_Ellinika',
     'Hebrew_Ivrit', 'Japanese_Nihongo', 'Korean_Hankuko']

cfd = nltk.ConditionalFreqDist(
          (lang, len(word))
          for lang in languages
          for word in udhr.words(lang + '-UTF8'))
cfd.tabulate(conditions=languages,
             samples=range(10), cumulative=True)

- 가깝고도 먼 나라 일본어와 비교해 볼까요...
    - 일본어가 전체적으로 단어 수가 적군요... 어휘 다양성이 우리말보다 낮은 것처럼 보입니다.

In [ ]:
cfd.tabulate(conditions=[ 'Korean_Hankuko', 'Japanese_Nihongo'],
             samples=range(10), cumulative=True)

###  Bigrams을 사용하여 무작위로 텍스트 생성
조건빈도분포를 사용하여 bigram(단어 쌍) 테이블을 만들 수 있습니다. 

- bigrams() 메소드는 단어 리스트를 입력받아 단어 쌍(consecutive word pair) 리스트를 만듭니다.
    - 실행 결과를 확인하기 위해서는 list() 메소드를 사용해야 함을 기억하세요. 

In [ ]:
sent = ['In', 'the', 'beginning', 'God', 'created', 'the', 'heaven',
       'and', 'the', 'earth', '.']
list(nltk.bigrams(sent))

### 무작위 텍스트 생성
1. 성경 창세기(genesis)로부터 모든 bigrams을 찾습니다. 
2. 조건빈도분포를 사용하여 seed word 의 이웃 단어로 가장 많이 나타난 단어를 찾습니다. (living --> creature --> ...)
3. 가장 많이 나타난 단어의 이웃 단어를 다시 찾습니다. 
4. 이 과정을 15번 반복하여, 15개 단어로 구성된 문장을 생성합니다.

In [ ]:
text = nltk.corpus.genesis.words('english-kjv.txt')
bigrams = nltk.bigrams(text)
cfd = nltk.ConditionalFreqDist(bigrams) 
#bigrams_list = list(bigrams)
#print(bigrams_list[:5])

단어 living 다음에 가장 많이 나온 단어는 creature. 

In [ ]:
cfd['living']

In [ ]:
cfd['living'].max()

- 문장의 시작 단어인 word를 기준으로 다음에 나올 단어를 빈도 분포에서 골라 문장을 생성합니다.
    - 15개 단어로 이루어진 문장을 생성

In [ ]:
def generate_model(cfdist, word, num=15):
    for i in range(num):
        print(word, end=' ')
        word = cfdist[word].max()

In [ ]:
generate_model(cfd, 'living')

## 파이썬 코드 재사용: 함수와 모듈

어휘 다양성(lexical diversity)를 파이썬 함수로 정의해 보겠습니다.

In [ ]:
def lexical_diversity(my_text_data):
    word_count = len(my_text_data)
    vocab_size = len(set(my_text_data))
    diversity_score = round(vocab_size / word_count, 4)
    return diversity_score

코딩 경험이 있으면 한 줄로 작성 가능하겠죠...

In [ ]:
def lexical_diversity_compact(text):
    return round(len(text) / len(set(text)), 4)

In [ ]:
from nltk.corpus import genesis
kjv = genesis.words('english-kjv.txt')
lexical_diversity(kjv)

- plural() 함수를 정의해 보겠습니다. 
    - 단수 명사를 입력으로 받아 복수형으로 변환합니다.
    - 완벽하지는 않지만 다양한 복수형 변환 특성을 반영하였습니다.

In [ ]:
def plural(word):
    if word.endswith('y'):
        return word[:-1] + 'ies'
    elif word[-1] in 'sx' or word[-2:] in ['sh', 'ch']:
        return word + 'es'
    elif word.endswith('an'):
        return word[:-2] + 'en'
    else:
        return word + 's'

In [ ]:
print(plural('fairy'))
print(plural('woman'))
print(plural('dish'))
print(plural('fan')) #에러 --> fans이 정답

### 모듈
- 모듈(module) : 변수 및 함수들을 파일에 저장.
- 패키지(package): 여러 개 모듈 집합(collection)
- 라이브러리(library) : 여러 개 패키지 집합.

# 어휘 자원(Lexical Resources)

lexicon(또는 lexical resource): 품사(POS) 등 연관 정보를 갖고 있는 단어 또는 구(phrase) 집합.
- lexicon은 텍스트 분석을 진행해 갈수록 상세한 정보를 갖게 됩니다. 
- 예: 텍스트에 사용된 단어와 빈도 수. 
- 예: 어떤 단어에 대해 concordance 메소드를 사용해 문장에서 어떻게 사용되었는지를 알게 됨. 
- lexicon = headword (또는 lemma) + 품사 + 정의(뜻). 
    - 예: saw(보았다) = saw + verb + 'past tense of see'. 
    - 예: saw(톱) = saw + noun + 'cutting instrument'

### 영단어사전 말뭉치
- 단어 사전 위치 : C:\Users\yshon\AppData\Roaming\Microsoft\Spelling\EN-US

- 영단어 사전과 텍스트에서 사용한 단어를 비교해서, 영단어 사전에 없는 단어만을 찾음. 
- 이 단어들은 현재 사용되지 않거나 철자가 틀린 단어들. 

In [ ]:
def unusual_words(text):
    text_vocab = set(w.lower() for w in text if w.isalpha())
    english_vocab = set(w.lower() for w in nltk.corpus.words.words())
    unusual = text_vocab - english_vocab #set difference
    return sorted(unusual)

In [ ]:
unused_words = unusual_words(nltk.corpus.gutenberg.words('austen-sense.txt'))
print(len(unused_words))
print(unused_words[:10]) #복수형, 과거형 등도 포함 

In [ ]:
unused_words = unusual_words(nltk.corpus.nps_chat.words())
print(len(unused_words))
print(unused_words[:10]) 

### stopwords (불용어)
- 'the'와 같은 불용어는 lexicon에서 거의 의미가 없기 때문에, 
    - 텍스트의 특징을 분석하거나 다른 텍스트와 구분할 때 오히려 방해만 됩니다.
- 대부분의 자연어 전처리과정에서 불용어는 가장 먼저 제거합니다.

In [ ]:
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
print(len(stopwords))
print(stopwords[:10]) 

어떤 텍스트에서 불용어가 어느 정도 비율을 차지하고 있는지 알아보겠습니다. 

In [ ]:
def content_fraction(text):
    stopwords = nltk.corpus.stopwords.words('english')
    content = [w for w in text if w.lower() not in stopwords]
    return round(len(content) / len(text), 4)

In [ ]:
content_fraction(nltk.corpus.reuters.words())

영단어사전을 활용하여 단어 퍼즐을 풀어볼까요...

In [ ]:
wordlist = nltk.corpus.words.words()
print(len(wordlist))

### 단어 퍼즐
1. 9개의 알파벳이 주어짐: 'egivrvonl' (알파벳 중복 허용, 3x3)
2. 반드시 포함해야 하는 알파벳 : 'r' (obligatory letter, 3x3 그리드 중앙에 놓인 알파벳)
3. 단어 길이 제한 : 6개 이상
4. Hint: FreqDist 메소드 사용 - 후보 단어에 속한 알파벳 빈도가 퍼즐에 포함된 알파벳의 빈도보다 작거나 같아야 함.

In [ ]:
obligatory = 'r'

puzzle_letters = nltk.FreqDist('egivrvonl')
puzzle_letters

In [ ]:
answers = [w for w in wordlist if len(w) >= 6 
                      and obligatory in w 
                      and nltk.FreqDist(w) <= puzzle_letters] 
print(len(answers))
print(answers)

### 이름 말뭉치
- 이름 말뭉치는 성별로 나뉜 8,000개의 first name입니다. 
    - 남성과 여성 이름이 각각 male.txt, female.txt에 저장되어 있습니다.

In [ ]:
names = nltk.corpus.names
print(names.fileids())

male_names = names.words('male.txt')
female_names = names.words('female.txt')
print(len(male_names))
print(len(female_names))

남성과 여성 이름으로 모두 포함된 이름을 찾아보겠습니다.

In [ ]:
both_names = [w for w in male_names if w in female_names]
print(len(both_names))
print(both_names[:10])

- 이름 맨마지막에 사용한 알파벳에 따라 남성 이름인지 여성 이름인지 분포를 조사해 보겠습니다.
    - 여성 이름은 마지막 알파벳이 'a','e'인 경우가 많습니다.
    - 반면, 남성은 'e', 'n', 's', 'y'의 경우가 많습니다. 

In [ ]:
cfd = nltk.ConditionalFreqDist(
          (fileid, name[-1])
           for fileid in names.fileids()
           for name in names.words(fileid))
cfd.plot()

### 발음 사전 : cmudict
 - 어휘 자원(lexicon)을 테이블 형태로 제공하는 대표적 경우입니다. 
 - 음성 합성에 사용할 목적으로 설계된 CMU(Carnegie Mellon University) 발음 사전이 대표적입니다.
 - 아래 예에서 fire는 2가지로 발음됩니다. 
     - 각 entry는 2개 부분(단어와 발음 기호)으로 나눌 수 있습니다.    
     - 첫 번째 fire의 발음 기호는 'F', 'AY1', 'ER0'인데, 각 음절에 대한 발음 기호를 phone이라 부릅니다.

In [ ]:
entries = nltk.corpus.cmudict.entries()
print(len(entries))
for entry in entries[42371:42379]:
    print(entry)

- 어떤 entry가 3개 음절로 발음되는(즉, phone이 3개) 경우를 찾습니다.
- 첫 번째 음절의 발음기호가 'P '이고, 세 번째 음절의 발음기호가 'T'인 단어를 찾습니다.

In [ ]:
for word, pron in entries:
    if len(pron) == 3: 
        ph1, ph2, ph3 = pron 
        if ph1 == 'P' and ph3 == 'T':
            print(word, ph2, end=' ')

마지막 음절이 'nicks'로 발음되는 모든 단어를 찾아보겠습니다. 

In [ ]:
syllable = ['N', 'IH0', 'K', 'S']
nicks_words = [word for word, pron in entries if pron[-4:] == syllable]
print(len(nicks_words))
print(nicks_words[:10])

- 발음과 단어 스펠링이 일치하지 않는 몇 가지 경우를 찾아보겠습니다. 
    - 발음은 M으로 끝나지만, 단어는 n으로 끝나는 단어입니다. 이 때 n은 묵음입니다.

In [ ]:
mn_words = [w for w, pron in entries if pron[-1] == 'M' and w[-1] == 'n']
print(len(mn_words))
print(mn_words)

- 발음과 단어 스펠링이 일치하지 않는 경우입니다.
- 발음은 N으로 시작하지만, 첫 번째 알파벳이 n이 아닙니다.

In [ ]:
nn_words = sorted(set(w[:2] for w, pron in entries if pron[0] == 'N' and w[0] != 'n'))
print(len(nn_words))
print(nn_words)

- 발음기호(phones)는 액센트를 나타내기 위해 0,1,2 숫자를 사용합니다. 
    - 이 숫자는 각각 primary stress (1), secondary stress (2) 및 stress 없음(0)을 나타냅니다.
    - stress 숫자로 단어의 발음 패턴을 정의하고, 이 stress 패턴을 갖는 단어를 찾아보겠습니다.

In [ ]:
def stress(pron):
    return [char for phone in pron 
                 for char in phone 
                 if char.isdigit()]

액센트 패턴이 0, 1, 0, 2, 0 인 단어를 찾습니다. 단어 뒤 쪽에 강한 액센트가 있습니다.

In [ ]:
phone_words = [w for w, pron in entries if stress(pron) == ['0', '1', '0', '2', '0']]
print(len(phone_words))
print(phone_words[:10])

액센트 패턴이 0, 2, 0, 1, 0 인 단어를 찾습니다. 단어 앞 쪽에 강한 액센트가 있습니다.

In [ ]:
phone_words = [w for w, pron in entries if stress(pron) == ['0', '2', '0', '1', '0']]
print(len(phone_words))
print(phone_words[:10])

3개의 발음기호를 갖는 단어 중에 첫 번째 발음이 'P'인 단어들을 모두 찾아보겠습니다.

In [ ]:
p3 = [(pron[0]+'-'+pron[2], word) 
      for (word, pron) in entries
      if pron[0] == 'P' and len(pron) == 3] 
print(p3[:10])

In [ ]:
cfd = nltk.ConditionalFreqDist(p3)
print(cfd['P-P'].most_common(5))
print(cfd['P-R'].most_common(5))
print(cfd['P-S'].most_common(5))
print(cfd.conditions()[:10])

- 조건빈도분포를 사용하여 원소 개수가 최소 10개 이상인 condition에 속한 단어들을 출력해 보겠습니다.
    - 한 줄에 최대 70개 단어 출력.

In [ ]:
for template in sorted(cfd.conditions()):
    if len(cfd[template]) > 10:
        words = sorted(cfd[template])
        wordstring = ' '.join(words)
        print(template, wordstring[:70] + "...")

- 사전 전체를 탐색하는 대신, 파이썬 dictionary 타입을 사용하여 특정 단어에 대해서만 찾아볼 수 있습니다. 
- 여기서 단어는 dictionary 객체의 key입니다.

In [ ]:
prondict = nltk.corpus.cmudict.dict()
print(prondict['fire'])

In [ ]:
#print(prondict['blog']) #key error 

- 단어 blog가 발음기호 사전에 빠져있어 key 에러가 발생합니다.
- 아래와 같이 키 blog 에 대한 값(value), 즉 발음 기호를 추가함으로써 해결할 수 있습니다. 
    - 그러나, NLTK 말뭉치에 이 키-값 쌍이 반영되지는 않습니다.

In [ ]:
prondict['blog'] = [['B', 'L', 'AA1', 'G']] 
prondict['blog']

문장에 속한 단어의 발음 기호를 순차적으로 조합해서 text-to-speech 기능을 구현할 수 있습니다. 

In [ ]:
text = ['natural', 'language', 'processing']
text_to_speech = [ph for w in text for ph in prondict[w][0]]
print(text_to_speech)

### 호환 단어 리스트(comparative wordlist)
- Swadesh wordlists: 24개국 언어에 대해 호환 가능한 207개 단어 리스트를 제공. 
- 언어는 2자리 표준 코드(ISO 639)로 구분. 

In [ ]:
from nltk.corpus import swadesh

print(len(swadesh.fileids()))
print(swadesh.fileids())
print(len(swadesh.words('en')))
print(swadesh.words('en')[:20])

In [ ]:
fr2en = swadesh.entries(['fr', 'en'])
print(fr2en[:10])

In [ ]:
translate = dict(fr2en)
print(translate['chien'])
print(translate['jeter'])

### 간단한 언어 번역기(단어 변환)
- German-English 와 Spanish-English 쌍에 대해, dict() 메소드를 사용해 dictionary 타입으로 변환합니다.
- update() 메소드를 호출하여 기존 translate에 이들 dictionary 객체를 추가합니다.

In [ ]:
de2en = swadesh.entries(['de', 'en'])    # German-English
es2en = swadesh.entries(['es', 'en'])    # Spanish-English
translate.update(dict(de2en))
translate.update(dict(es2en))
print(translate['Hund'])
print(translate['perro'])

게르만(Germanic) 언어와 로마(Romance) 언어에 속한 단어들을 비교해 보겠습니다.

In [ ]:
languages = ['en', 'de', 'nl', 'es', 'fr', 'pt', 'la']
for i in range(139, 145):
    print(swadesh.entries(languages)[i])